In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib as mpl

import matplotlib.pyplot as plt
import platform
if platform.system() == "Windows":
    plt.rcParams['font.family'] = ['Times new roman',] # windows->Heiti TC
else:
    plt.rcParams['font.family'] = ['Heiti TC',] # windows->Heiti TC
    
plt.rcParams['axes.unicode_minus'] = False # windows->simhei
plt.rcParams['figure.dpi'] = 200

import geopandas as gpd
import shapely
from shapely.geometry import Point,Polygon,LineString,MultiLineString,MultiPoint,MultiPolygon

import pickle
from tqdm import tqdm
from glob import glob
import joblib

import seaborn as sns

In [2]:
import os

import platform
if platform.system() == "Windows":
    os.chdir(r'H:\BaiduSyncdisk\DR.MENG-Full\Y2024-002-DT-NANJING-ACCESSIBILITY-MAUP')
else:
    os.chdir(r'/Volumes/SANDISK/DR.MENG')

# 读取数据

In [3]:
# od_walk_buildings  od_pt_buildings od_nav_buildings od_cycle_buildings

In [4]:
file_name = 'od_walk_buildings' 
time_threshold = 900 # 900 1800

In [5]:
od_nav = pd.read_csv(f'./data_to_cal_accessibility/{file_name}.csv')

In [6]:
od_nav.tail(1)

,od_id,distance,duration,building_id,x_res,y_res,name,park_id,x_parks,y_parks
10276168,od_4841371,5557,4763,b_132720,118.738063,32.052189,北极阁公园,p_11,118.783473,32.062453


In [7]:
# od_nav.rename(columns={'net_id':'building_id'}, inplace=True)

In [8]:
len(od_nav)

10276169

In [9]:
area_pop = joblib.load('./od_request_data_buildings/area_pop.joblib')

In [10]:
area = area_pop['area']
pop = area_pop['pop']

In [11]:
area.head(1)

,park_id,park_area
0,p_0,132881


In [12]:
pop.head(1)

,building_id,building_pop
0,b_0,87


In [13]:
res = od_nav.merge(area, on='park_id',how='left').\
    merge(pop, on='building_id',how='left')

In [14]:
res = res.fillna(0)

In [15]:
res

,od_id,distance,duration,building_id,x_res,y_res,name,park_id,x_parks,y_parks,park_area,building_pop
0,od_8221176,5711,4895,b_130167,118.759773,32.079866,朝天宫公园,p_132,118.768702,32.035746,94086,0.0
1,od_9248823,5235,4487,b_31673,118.777357,32.032726,月牙湖公园,p_23,118.825773,32.032019,673374,6.0
2,od_3082941,4389,3762,b_5882,118.789240,31.927500,凤凰台公园（百家湖广场）,p_52,118.815414,31.936851,60024,111.0
3,od_5138235,2519,2159,b_132980,118.809417,32.027203,西安门遗址公园,p_17,118.800653,32.041573,10317,70.0
4,od_6165882,2831,2426,b_99936,118.729463,32.055453,石头城公园（南京国防园）,p_102,118.751018,32.050519,145355,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10276164,od_4841367,5557,4763,b_132720,118.738063,32.052189,北极阁公园,p_11,118.783425,32.062465,121334,4.0
10276165,od_4841368,5563,4768,b_132720,118.738063,32.052189,北极阁公园,p_11,118.785818,32.060344,121334,4.0
10276166,od_4841369,5513,4725,b_132720,118.738063,32.052189,北极阁公园,p_11,118.783051,32.062195,121334,4.0
10276167,od_4841370,5949,5099,b_132720,118.738063,32.052189,北极阁公园,p_11,118.789184,32.060668,121334,4.0


In [16]:
# 步行10分钟，15分钟，30分钟

# 600s == 10 mins

## 2SFCA first step

In [17]:
10*60,15*60,30*60

# 10 分钟 15分钟 30分钟

(600, 900, 1800)

In [18]:
res.head(2)

,od_id,distance,duration,building_id,x_res,y_res,name,park_id,x_parks,y_parks,park_area,building_pop
0,od_8221176,5711,4895,b_130167,118.759773,32.079866,朝天宫公园,p_132,118.768702,32.035746,94086,0.0
1,od_9248823,5235,4487,b_31673,118.777357,32.032726,月牙湖公园,p_23,118.825773,32.032019,673374,6.0


In [19]:
resx = res.sort_values(by=['building_id','park_id','duration'])
len(resx)

10276169

In [20]:
resx = resx.drop_duplicates(subset=['building_id','park_id'],keep='first')
len(resx)  # 获取了每一幢住宅至公园的最短耗时，

2745586

In [21]:
def get_G(x):
    threshold = time_threshold
    if x <= threshold:
        G = (np.exp(-0.5 * (x/threshold)**2) - np.exp(-0.5)) / (1 - np.exp(-0.5))
        return G
    else:
        return 0

def get_Rj(x):
    x = x.reset_index()
    Sj = x['park_area'][0]

    dt = 0
    for i in range(len(x)):
        if x['duration'][i] > 1:
            vl = x['building_pop'][i] * get_G(x['duration'][i])
            dt = dt + vl
        else:
            dt = 0
    # print(dt)
    if dt == 0:
        return 0
    else:
        return Sj/dt

In [22]:
resx[resx['park_id'] == 'p_8'].sort_values(by='duration')

,od_id,distance,duration,building_id,x_res,y_res,name,park_id,x_parks,y_parks,park_area,building_pop
288004,od_28419,53,45,b_33740,118.757577,32.028712,南湖公园,p_8,118.757111,32.028639,87331,30.0
688775,od_68131,58,49,b_77512,118.757303,32.028835,南湖公园,p_8,118.757111,32.028639,87331,4.0
688711,od_68124,73,62,b_77509,118.758897,32.031051,南湖公园,p_8,118.758127,32.030900,87331,0.0
329793,od_32686,75,64,b_39466,118.758897,32.031069,南湖公园,p_8,118.758127,32.030900,87331,0.0
688568,od_68107,78,66,b_77500,118.757577,32.028748,南湖公园,p_8,118.757111,32.028639,87331,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1215807,od_120507,8385,7187,b_138375,118.707356,32.047423,南湖公园,p_8,118.757111,32.028639,87331,168.0
1215909,od_120517,8392,7193,b_138380,118.708348,32.045376,南湖公园,p_8,118.757111,32.028639,87331,168.0
1215893,od_120515,8400,7200,b_138379,118.707570,32.045694,南湖公园,p_8,118.757111,32.028639,87331,168.0
1215766,od_120503,8471,7260,b_138373,118.706782,32.046554,南湖公园,p_8,118.757111,32.028639,87331,168.0


In [23]:
resx[resx['park_id'] == 'p_8'].groupby(by='park_id').apply(get_Rj).reset_index().fillna(0)

,park_id,0
0,p_8,3.089308


In [24]:
# res[res['building_id'] == 'b_0'].groupby(by='park_id').apply(get_Rj).reset_index().fillna(0)

In [25]:
park_vk = resx.groupby(by='park_id').apply(get_Rj).reset_index().fillna(0)

In [26]:
# park_vk.sort_values(by=0)

In [27]:
park_vk.columns=['park_id','Rj']

In [28]:
park_vk.head()

,park_id,Rj
0,p_0,6.274865
1,p_1,2.376337
2,p_10,75.612003
3,p_100,59.404823
4,p_101,145.469963


In [29]:
park_vk.sort_values(by='Rj') # RJ排序

,park_id,Rj
121,p_98,0.000000
78,p_44,0.000000
29,p_125,0.000000
30,p_126,0.000000
97,p_61,0.000000
...,...,...
8,p_106,5795.975149
74,p_40,9984.432638
96,p_60,10475.931125
92,p_57,18645.215133


In [30]:
# park_vk[park_vk['Rj'] > 0]

In [31]:
res2 = resx.copy()

In [32]:
res2.head(1)

,od_id,distance,duration,building_id,x_res,y_res,name,park_id,x_parks,y_parks,park_area,building_pop
1217521,od_120684,10719,9187,b_0,118.729455,31.994567,青奥森林公园,p_10,118.691604,32.00272,273225,87.0


In [33]:
res2 = res2.merge(park_vk, on='park_id')

In [34]:
res2.head(1)

,od_id,distance,duration,building_id,x_res,y_res,name,park_id,x_parks,y_parks,park_area,building_pop,Rj
0,od_120684,10719,9187,b_0,118.729455,31.994567,青奥森林公园,p_10,118.691604,32.00272,273225,87.0,75.612003


## 2SFCA 2 step

In [35]:
def get_A(x):
    x = x.reset_index()
    dt = 0
    for i in range(len(x)):
        vl = x['Rj'][i] * get_G(x['duration'][i])
        dt = dt + vl
    return dt

In [36]:
res2_vl = res2.groupby(by='building_id').apply(get_A)

In [37]:
res2_vl

building_id
b_0         0.000000
b_1         1.111760
b_10        0.000000
b_100      35.572087
b_1000      1.491775
             ...    
b_99995     0.000000
b_99996     0.000000
b_99997     0.000000
b_99998     0.000000
b_99999     0.000000
Length: 139409, dtype: float64

In [38]:
res2_vl.sort_values()

building_id
b_0            0.000000
b_45510        0.000000
b_45509        0.000000
b_45508        0.000000
b_45507        0.000000
               ...     
b_66483    10944.510846
b_66481    12484.423182
b_66484    13030.588176
b_35089    73968.878678
b_35090    93046.133172
Length: 139409, dtype: float64

In [39]:
res2_vl = res2_vl.reset_index()

In [40]:
res2_vl.columns = ['building_id','access_val']

# 空间连接

In [41]:
buildings = gpd.read_file('./od_request_data_buildings/buildings_with_id.geojson')

In [42]:
buildings.head(1)

,floors,building_id,geometry
0,2.0,b_0,"POLYGON ((118.72926 31.99482, 118.72953 31.994..."


In [43]:
buildings = buildings.merge(res2_vl)

In [44]:
buildings

,floors,building_id,geometry,access_val
0,2.0,b_0,"POLYGON ((118.72926 31.99482, 118.72953 31.994...",0.000000
1,2.0,b_1,"POLYGON ((118.72762 31.99548, 118.72758 31.995...",1.111760
2,2.0,b_2,"POLYGON ((118.72758 31.99552, 118.72762 31.995...",1.233924
3,2.0,b_3,"POLYGON ((118.72744 31.9957, 118.7275 31.99564...",1.339204
4,2.0,b_4,"POLYGON ((118.72751 31.99613, 118.72747 31.996...",1.144405
...,...,...,...,...
139404,20.0,b_139986,"POLYGON ((118.8248 31.90135, 118.8248 31.90118...",0.000000
139405,20.0,b_139987,"POLYGON ((118.82475 31.90083, 118.82475 31.900...",0.000000
139406,20.0,b_139988,"POLYGON ((118.82389 31.90074, 118.82389 31.900...",0.000000
139407,20.0,b_139989,"POLYGON ((118.82394 31.90022, 118.82394 31.900...",0.000000


In [45]:
buildings.sort_values(by='access_val')

,floors,building_id,geometry,access_val
0,2.0,b_0,"POLYGON ((118.72926 31.99482, 118.72953 31.994...",0.000000
80091,8.0,b_80412,"POLYGON ((118.80169 31.9999, 118.80171 31.9999...",0.000000
80090,8.0,b_80411,"POLYGON ((118.80144 31.99983, 118.80145 31.999...",0.000000
80089,8.0,b_80410,"POLYGON ((118.80154 31.99987, 118.80156 31.999...",0.000000
80088,6.0,b_80409,"POLYGON ((118.80298 32.00064, 118.80316 32.000...",0.000000
...,...,...,...,...
66216,8.0,b_66483,"POLYGON ((119.04868 32.06242, 119.0487 32.0623...",10944.510846
66214,2.0,b_66481,"POLYGON ((119.04855 32.0623, 119.04862 32.0619...",12484.423182
66217,8.0,b_66484,"POLYGON ((119.04862 32.06195, 119.04882 32.061...",13030.588176
34893,2.0,b_35089,"POLYGON ((118.80883 32.05991, 118.80888 32.059...",73968.878678


In [46]:
buildings.to_file(f'./data_access_geojson/{file_name}_{time_threshold}.geojson')